Read in TSV file

In [165]:
def get_id(line):
    return "_".join(map(str, [line['chromosome_name'], line['start'], line['stop']]))

In [166]:
import pandas as pd
table = pd.read_table("A1.tsv")
table['id']=table.apply(get_id, axis=1)
table = table.set_index('id')

ref_cols = ['tumor.rcnt.llr3_ref','adrenalmet.rcnt.llr3_ref', 'livermet.rcnt.llr3_ref', 'lungmet.rcnt.llr3_ref', 'spinalmet.rcnt.llr3_ref']
var_cols = ['tumor.rcnt.llr3_var','adrenalmet.rcnt.llr3_var', 'livermet.rcnt.llr3_var', 'lungmet.rcnt.llr3_var', 'spinalmet.rcnt.llr3_var']

#breast,adrenal,liver,lung,spinal
cols = ['breast', 'adrenal', 'liver', 'lung', 'spinal']
table = table[['cluster']+ref_cols+var_cols]
table.columns = ['cluster']+['ref-'+c for c in cols] + ['var-'+c for c in cols]
table.head()

,cluster,ref-breast,ref-adrenal,ref-liver,ref-lung,ref-spinal,var-breast,var-adrenal,var-liver,var-lung,var-spinal
id,,,,,,,,,,,
1_110359957_110359957,7,150,158,104,151,142,4,2,20,0,0
6_48902663_48902663,6,189,150,97,147,178,0,27,0,1,0
6_49276467_49276467,9,117,91,144,85,63,1,0,0,0,17
6_50173697_50173697,9,179,117,92,126,115,0,0,0,0,30
6_50179508_50179508,6,141,113,98,113,128,0,26,0,0,0


In [167]:
ctable = table.groupby('cluster').mean()

Get intervals

In [168]:
import numpy
from scipy.stats import beta
from scipy.stats import norm

def binomial_hpdr(n, N, pct, a=1, b=1, n_pbins=1e3):
    """
    Function computes the posterior mode along with the upper and lower bounds of the
    **Highest Posterior Density Region**.

    Parameters
    ----------
    n: number of successes 
    N: sample size 
    pct: the size of the confidence interval (between 0 and 1)
    a: the alpha hyper-parameter for the Beta distribution used as a prior (Default=1)
    b: the beta hyper-parameter for the Beta distribution used as a prior (Default=1)
    n_pbins: the number of bins to segment the p_range into (Default=1e3)

    Returns
    -------
    A tuple that contains the mode as well as the lower and upper bounds of the interval
    (mode, lower, upper)

    """
    # fixed random variable object for posterior Beta distribution
    rv = beta(n+a, N-n+b)
    # determine the mode and standard deviation of the posterior
    stdev = rv.stats('v')**0.5
    mode = (n+a-1.)/(N+a+b-2.)
    # compute the number of sigma that corresponds to this confidence
    # this is used to set the rough range of possible success probabilities
    n_sigma = numpy.ceil(norm.ppf( (1+pct)/2. ))+1
    # set the min and max values for success probability 
    max_p = mode + n_sigma * stdev
    if max_p > 1:
        max_p = 1.
    min_p = mode - n_sigma * stdev
    if min_p > 1:
        min_p = 1.
    # make the range of success probabilities
    p_range = numpy.linspace(min_p, max_p, n_pbins+1)
    # construct the probability mass function over the given range
    if mode > 0.5:
        sf = rv.sf(p_range)
        pmf = sf[:-1] - sf[1:]
    else:
        cdf = rv.cdf(p_range)
        pmf = cdf[1:] - cdf[:-1]
    # find the upper and lower bounds of the interval 
    sorted_idxs = numpy.argsort( pmf )[::-1]
    cumsum = numpy.cumsum( numpy.sort(pmf)[::-1] )
    j = numpy.argmin( numpy.abs(cumsum - pct) )
    upper = p_range[ (sorted_idxs[:j+1]).max()+1 ]
    lower = p_range[ (sorted_idxs[:j+1]).min() ]    

    return (mode, lower, upper)

In [178]:


def get_ub(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], 0.95)
    return v[2]
    

def get_lb(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], 0.95)
    mval = v[1]
    #if mval < 0.01: mval = 0
    return mval

ctable = table.groupby('cluster').mean()
for sam in cols:
    ctable['ub-'+sam]= ctable.apply(get_ub, args=[sam], axis=1)
    ctable['lb-'+sam]= ctable.apply(get_lb, args=[sam], axis=1)
    ctable[sam]= ctable.apply(get_mean, args=[sam], axis=1)
    

In [170]:
ctable

,ref-breast,ref-adrenal,ref-liver,ref-lung,ref-spinal,var-breast,var-adrenal,var-liver,var-lung,var-spinal,ub-breast,lb-breast,ub-adrenal,lb-adrenal,ub-liver,lb-liver,ub-lung,lb-lung,ub-spinal,lb-spinal
cluster,,,,,,,,,,,,,,,,,,,,
1,131.400000,114.200000,110.200000,122.400000,139.600000,53.400000,61.400000,87.400000,29.400000,34.000000,0.356791,0.227098,0.421674,0.281930,0.511749,0.374339,0.261084,0.136446,0.258981,1.417423e-01
2,181.933333,108.766667,95.000000,152.400000,178.566667,10.733333,76.233333,75.200000,2.333333,0.400000,0.094195,0.029065,0.483605,0.343125,0.516559,0.368919,0.043891,0.002811,0.020590,1.374663e-05
3,138.960000,175.240000,179.800000,124.960000,137.220000,41.400000,0.140000,0.160000,21.840000,33.260000,0.294304,0.172452,0.018317,-0.000027,0.018082,-0.000029,0.212035,0.097705,0.258708,1.405721e-01
4,182.362500,137.250000,136.875000,152.112500,177.437500,6.075000,40.925000,45.312500,0.587500,0.125000,0.064307,0.012883,0.294861,0.172242,0.314515,0.189990,0.026139,0.000019,0.017968,-6.687842e-07
5,144.156250,173.843750,187.625000,135.093750,175.000000,41.781250,0.250000,0.343750,22.500000,0.812500,0.288034,0.168882,0.019605,-0.000002,0.019076,0.000009,0.202835,0.094286,0.024903,1.144234e-04
6,185.000000,138.043478,161.521739,150.652174,174.782609,0.086957,38.434783,0.217391,0.173913,0.086957,0.016862,-0.000029,0.282265,0.161486,0.020700,-0.000021,0.021680,-0.000030,0.017831,-3.012584e-05
7,184.300000,178.466667,149.633333,156.433333,173.733333,5.066667,0.100000,41.866667,0.133333,0.000000,0.056770,0.009626,0.017607,-0.000029,0.280442,0.164133,0.020418,-0.000029,0.016974,0.000000e+00
8,154.000000,180.571429,214.785714,153.142857,186.285714,38.785714,0.071429,0.071429,0.928571,0.000000,0.261346,0.148977,0.017098,-0.000010,0.014413,-0.000010,0.029615,0.000209,0.015849,0.000000e+00
9,188.015385,182.169231,172.707692,153.261538,142.184615,0.153846,0.123077,0.046154,0.184615,32.753846,0.017229,-0.000025,0.017484,-0.000011,0.017584,-0.000012,0.021436,-0.000007,0.249262,1.343892e-01


In [171]:
def get_ub(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], 0.95)
    return v[2]
    

def get_lb(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], 0.95)
    mval = v[1]
    if mval < 0.01: mval = 0
    return mval

def get_mean(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], 0.95)
    mval = v[0]
    return mval

ctable_cutoff = table.groupby('cluster').mean()
for sam in cols:
    ctable_cutoff['ub-'+sam]= ctable.apply(get_ub, args=[sam], axis=1)
    ctable_cutoff['lb-'+sam]= ctable.apply(get_lb, args=[sam], axis=1)
    ctable_cutoff[sam]= ctable.apply(get_mean, args=[sam], axis=1)

In [172]:

def get_ub(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], 0.95)
    return v[2]
    

def get_lb(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], 0.95)
    mval = v[1]
    if mval < 0.01: mval = 0
    return mval

def get_vaf(row, sam):
    return float(row['var-'+sam])/float(row['var-'+sam]+row['ref-'+sam])

#ctable_cutoff = table.groupby('cluster').mean()
vafs = pd.DataFrame()
for sam in cols:
    vafs[sam] = table.apply(get_vaf, args=[sam], axis=1)
vafs['cluster'] = table['cluster']
stds = vafs.groupby('cluster').std()

stds.columns = ['std-'+sam for sam in cols]
print vafs.groupby('cluster').mean().head()
print stds.head()

           breast   adrenal     liver      lung    spinal
cluster                                                  
1        0.283901  0.330158  0.438077  0.191010  0.200579
2        0.057373  0.405404  0.443312  0.016071  0.002040
3        0.228888  0.000710  0.000846  0.149339  0.194938
4        0.033383  0.228666  0.244060  0.003665  0.000774
5        0.225846  0.001270  0.001753  0.142480  0.004489
         std-breast  std-adrenal  std-liver  std-lung  std-spinal
cluster                                                          
1          0.049420     0.101859   0.032086  0.048586    0.047626
2          0.019771     0.061097   0.040163  0.012241    0.003952
3          0.041484     0.002502   0.002147  0.031672    0.039837
4          0.018849     0.074970   0.077713  0.005259    0.002866
5          0.041009     0.002493   0.003745  0.035509    0.005620


In [173]:
cvafs = vafs.groupby('cluster').mean()
cvafs = pd.merge(cvafs, stds, left_index=True, right_index=True)


def get_ub(row, sam):
    return min(row[sam]+2*(row['std-'+sam]), 1.)

def get_lb(row, sam):
    if row[sam] < 0.01: return max(row[sam]-2*(row['std-'+sam]), 0)
    else: return max(row[sam]-2*(row['std-'+sam]), 0.01)

for sam in cols:
    cvafs['ub-'+sam] = cvafs.apply(get_ub, args=[sam], axis=1)
    cvafs['lb-'+sam] = cvafs.apply(get_lb, args=[sam], axis=1)


In [174]:
cvafs

,breast,adrenal,liver,lung,spinal,std-breast,std-adrenal,std-liver,std-lung,std-spinal,ub-breast,lb-breast,ub-adrenal,lb-adrenal,ub-liver,lb-liver,ub-lung,lb-lung,ub-spinal,lb-spinal
cluster,,,,,,,,,,,,,,,,,,,,
1,0.283901,0.330158,0.438077,0.191010,0.200579,0.049420,0.101859,0.032086,0.048586,0.047626,0.382741,0.185061,0.533875,0.126440,0.502248,0.373906,0.288182,0.093837,0.295830,0.105328
2,0.057373,0.405404,0.443312,0.016071,0.002040,0.019771,0.061097,0.040163,0.012241,0.003952,0.096915,0.017831,0.527598,0.283209,0.523637,0.362987,0.040553,0.010000,0.009945,0.000000
3,0.228888,0.000710,0.000846,0.149339,0.194938,0.041484,0.002502,0.002147,0.031672,0.039837,0.311856,0.145920,0.005715,0.000000,0.005141,0.000000,0.212683,0.085996,0.274612,0.115264
4,0.033383,0.228666,0.244060,0.003665,0.000774,0.018849,0.074970,0.077713,0.005259,0.002866,0.071081,0.010000,0.378606,0.078726,0.399486,0.088634,0.014184,0.000000,0.006505,0.000000
5,0.225846,0.001270,0.001753,0.142480,0.004489,0.041009,0.002493,0.003745,0.035509,0.005620,0.307864,0.143828,0.006256,0.000000,0.009243,0.000000,0.213497,0.071463,0.015730,0.000000
6,0.000549,0.223056,0.001972,0.001001,0.000542,0.001829,0.044274,0.005134,0.002653,0.001854,0.004207,0.000000,0.311604,0.134507,0.012240,0.000000,0.006307,0.000000,0.004249,0.000000
7,0.026130,0.000621,0.217263,0.000651,0.000000,0.011403,0.002507,0.042290,0.002048,0.000000,0.048935,0.010000,0.005636,0.000000,0.301843,0.132684,0.004747,0.000000,0.000000,0.000000
8,0.202896,0.000378,0.000399,0.007029,0.000000,0.047994,0.001414,0.001493,0.013468,0.000000,0.298885,0.106907,0.003206,0.000000,0.003385,0.000000,0.033965,0.000000,0.000000,0.000000
9,0.000902,0.000894,0.000275,0.001013,0.190299,0.002274,0.003268,0.001301,0.003201,0.044461,0.005451,0.000000,0.007429,0.000000,0.002877,0.000000,0.007415,0.000000,0.279221,0.101377


In [175]:


rows = ["5 #m\n9 #n\n#sample_index\tsample_label\tcharacter_label\tcharacter_index\tvaf_lb\tvaf_mean\tvaf_ub\tx\ty\tmu\n",]

def print_char(row, sam):
    #print "\t".join(map(str,[i, sam, row.name-1, 'cluster_'+str(row.name), row['lb-'+sam], row[sam], row['lb-'+sam], 1, 1, 1]))
    return "\t".join(map(str,[i, sam, row.name-1, 'cluster_'+str(row.name), row['lb-'+sam], row[sam], row['ub-'+sam], 1, 1, 1]))+"\n"

for i, sam in enumerate(cols):
    rows += list(cvafs.apply(print_char, args=[sam], axis=1))

with open("A1_spruce_vafs.tsv", 'w') as f:
    for line in rows:
        f.write(line)

        



In [176]:
rows = ["5 #m\n9 #n\n#sample_index\tsample_label\tcharacter_label\tcharacter_index\tvaf_lb\tvaf_mean\tvaf_ub\tx\ty\tmu\n",]

def print_char(row, sam):
    #print "\t".join(map(str,[i, sam, row.name-1, 'cluster_'+str(row.name), row['lb-'+sam], row[sam], row['lb-'+sam], 1, 1, 1]))
    return "\t".join(map(str,[i, sam, row.name-1, 'cluster_'+str(row.name), row['lb-'+sam], row[sam], row['ub-'+sam], 1, 1, 1]))+"\n"

for i, sam in enumerate(cols):
    rows += list(ctable_cutoff.apply(print_char, args=[sam], axis=1))

with open("A1_spruce_betas.tsv", 'w') as f:
    for line in rows:
        f.write(line)


In [179]:
rows = ["5 #m\n9 #n\n#sample_index\tsample_label\tcharacter_label\tcharacter_index\tvaf_lb\tvaf_mean\tvaf_ub\tx\ty\tmu\n",]

def print_char(row, sam):
    #print "\t".join(map(str,[i, sam, row.name-1, 'cluster_'+str(row.name), row['lb-'+sam], row[sam], row['lb-'+sam], 1, 1, 1]))
    return "\t".join(map(str,[i, sam, row.name-1, 'cluster_'+str(row.name), row['lb-'+sam], row[sam], row['ub-'+sam], 1, 1, 1]))+"\n"

for i, sam in enumerate(cols):
    rows += list(ctable.apply(print_char, args=[sam], axis=1))

with open("A1_spruce_betas_nocutoff.tsv", 'w') as f:
    for line in rows:
        f.write(line)
